In [ ]:
from functions import cholesky_tsqr, direct_tsqr, indirect_tsqr


In [2]:
import time
from dask.distributed import wait
import numpy as np


In [ ]:
from dask.distributed import Client, LocalCluster

# Example: 4 workers, 1 thread each
cluster = LocalCluster(n_workers=4, threads_per_worker=1)
client = Client(cluster)

print("Dashboard:", client.dashboard_link)


Dashboard: http://127.0.0.1:8787/status


2025-09-14 16:17:27,418 - tornado.application - ERROR - Uncaught exception GET /status/ws (127.0.0.1)
HTTPServerRequest(protocol='http', host='127.0.0.1:8787', method='GET', uri='/status/ws', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "/home/riccorte/miniconda3/envs/dask-env/lib/python3.11/site-packages/tornado/websocket.py", line 965, in _accept_connection
    open_result = handler.open(*handler.open_args, **handler.open_kwargs)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/riccorte/miniconda3/envs/dask-env/lib/python3.11/site-packages/tornado/web.py", line 3375, in wrapper
    return method(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/riccorte/miniconda3/envs/dask-env/lib/python3.11/site-packages/bokeh/server/views/ws.py", line 149, in open
    raise ProtocolError("Token is expired. Configure the app with a larger value for --session-token-expiration if necessary")
bok

In [ ]:
# check if everything went smoothly
print(client)
print(cluster)


<Client: 'tcp://127.0.0.1:38757' processes=4 threads=4, memory=5.79 GiB>
LocalCluster(b41228c5, 'tcp://127.0.0.1:38757', workers=4, threads=4, memory=5.79 GiB)


In [ ]:
import dask.array as da
import dask
import numpy as np
import time
from scipy.linalg import solve_triangular 
from sklearn.datasets import fetch_california_housing

# Download California Housing dataset
data = fetch_california_housing(as_frame=True)

# Convert features into Dask Array (it's a matrix).
n_partition = 3        # number of partition in memory. We have 4 VMS (1 master + 3 workers), so let's start with just 3 partitions
length_partition = data.data.shape[0] // n_partition
X_da = da.from_array(data.data.values, chunks=(length_partition, data.data.shape[1]))

print("Number of Dask partitions:",  X_da.npartitions) 
print("Length of each partition:", length_partition, "rows")
print("Length of the whole dataset:", data.data.shape[0], "rows")


Number of Dask partitions: 3
Length of each partition: 6880 rows
Length of the whole dataset: 20640 rows


In [ ]:
#X_cached = X_da.rechunk((1_000_000, -1)).persist()

print(X_da)
A0 = X_da.rechunk((1_000_000, -1))


dask.array<array, shape=(20640, 8), dtype=float64, chunksize=(6880, 8), chunktype=numpy.ndarray>


In [7]:
import time
import numpy as np
from dask.distributed import wait
from time import sleep

def ensure_scaled(target, timeout=60):
    """Request scaling and wait until at least `target` workers are available.
       Returns the actual number of workers available (may be > target)."""
    cluster.scale(target)
    t0 = time.time()
    while time.time() - t0 < timeout:
        info = client.scheduler_info()
        nworkers = len(info.get("workers", {}))
        if nworkers >= target:
            return nworkers
        sleep(0.5)
    # timeout expired — return current state
    return len(client.scheduler_info().get("workers", {}))

def print_workers_info():
    info = client.scheduler_info().get("workers", {})
    for addr, w in info.items():
        print(addr, "nthreads:", w.get("nthreads"), "processing:", len(w.get("processing", {})),
              "memory:", w.get("memory_limit"))

def run_once_indirect(A):
    t0 = time.time()
    Q, R = indirect_tsqr(A)
    Q = Q.persist()
    try:
        wait(Q, timeout=120)    # avoid hanging forever
    except Exception as e:
        print("wait(Q) raised:", repr(e))
        # show scheduler state for debugging
        print_workers_info()
        raise
    t1 = time.time()
    client.cancel(Q)
    return t1 - t0

def time_vs_workers(workers_list, reps=2):
    results = {}
    for w in workers_list:
        actual = ensure_scaled(w, timeout=120)
        print(f"\nRequested {w}, actual available {actual}")
        print_workers_info()

        # adapt chunking to available workers so tasks can be distributed
        # here we assume A0 has its first axis as the partitioning axis
        try:
            n_rows = int(A0.shape[0])
            rows_per_chunk = (n_rows + actual - 1) // actual  # ceil division
            A = A0.rechunk((rows_per_chunk, -1)).persist()
            wait(A, timeout=120)
        except Exception as e:
            print("Error persisting A after rechunk:", repr(e))
            print_workers_info()
            # Try a simpler test to see if workers can do anything:
            test = client.submit(lambda x: x+1, 1)
            print("simple submit result:", test.result(timeout=10))
            raise

        # Warmup + measures
        _ = run_once_indirect(A)
        times = [run_once_indirect(A) for _ in range(reps)]
        results[w] = (float(np.mean(times)), float(np.std(times)))
        print(f"w={w}: mean={results[w][0]:.3f}s  std={results[w][1]:.3f}s")

        # cleanup
        client.cancel(A)
        del A
        # optionally scale down between runs:
        # cluster.scale(0)
        # sleep(2)

    return results


In [ ]:
# ESEMPIO: misura con 1, 2, 4, 8 worker (se la macchina lo consente)
res = time_vs_workers([1,2,3, 4], reps=2)
print("\nRisultati:", res)



Requested 1, actual available 4
tcp://127.0.0.1:36993 nthreads: 1 processing: 0 memory: 1554740224
w=1: mean=0.237s  std=0.049s

Requested 2, actual available 2
tcp://127.0.0.1:36993 nthreads: 1 processing: 0 memory: 1554740224
tcp://127.0.0.1:41841 nthreads: 1 processing: 0 memory: 1554740224
w=2: mean=0.113s  std=0.001s

Requested 3, actual available 3
tcp://127.0.0.1:36993 nthreads: 1 processing: 0 memory: 1554740224
tcp://127.0.0.1:41841 nthreads: 1 processing: 0 memory: 1554740224
tcp://127.0.0.1:46247 nthreads: 1 processing: 0 memory: 1554740224
w=3: mean=0.136s  std=0.021s

Requested 4, actual available 4
tcp://127.0.0.1:36993 nthreads: 1 processing: 0 memory: 1554740224
tcp://127.0.0.1:41841 nthreads: 1 processing: 0 memory: 1554740224
tcp://127.0.0.1:42393 nthreads: 1 processing: 0 memory: 1554740224
tcp://127.0.0.1:46247 nthreads: 1 processing: 0 memory: 1554740224
w=4: mean=0.197s  std=0.024s

Risultati: {1: (0.23735105991363525, 0.04942309856414795), 2: (0.1134823560714721